### CMSE 202 Final Project, Section 003

Benjamin Laubach, Sanskriti Verma, Carrie Neilsen, Tori Fallon, Ella Kovach and Jason Williams

## Research Question

How do varying factors affect moss gorth in the environment?

## Background and Motivation



Our project is an Agent Based Model (ABM) similar to those that we have focused on in class, we took inspiration from the environment class from homework 3 as well as the bacteria model earlier in the semester. When thinking about how to approach this project we discussed the basic needs of the model which included creating a board and then having a class for the growth of the moss. From some research we arrived at the conclusion that weeds are a direct competitor to moss growth as well as trees, these were two features we wanted to include in our classes as well as some others which included the level or moisture in the ground so whethere the moss was growing on wet land or dry land. 

...

* think complexity

## Methodology

In [ ]:
#Insert Code

## Discussion and Conclusion

### Overview of the classes 

The moss class is similar in ways to the animal class from homework three, inspiration was taken from the age functions and the check if dead function. We set an age limit of 5 iterations for each moss, the check if dead function takes this age limit and compares it to each moss on the board and returns a true or false value. **WORKING ON CODE TO REMOVE THE MOSS IF DEAD** From here we have a replicate function which randomises the adjacent cells on the board, after picking a random cell there is a code which checks the cell type whether a wet or dry land cell type as this affects the growth rate of the moss. Once the code has checked through all of the variables a moss replicate is spawned in a random adjacent cell...



After doing some research into what affects moss growth we decided to include two other classes, the first being the weed class which adds a competitor alongside the moss. This was the first layer of complexity that we added to our model. The init function initialises the same inputs as the moss class, then uses the moss class to reinitialise these input in the new class. In these case the moss class is acting as a parent or base class and the weed class is acting as a child or derived class. The replicate class is the same for both the moss and weed class as mentioned above it checks through a list of parameter to randomise where the new weed will spawn from...


The second layer of complexity we included was by creating an environment class, the aim of this class was to create the addition of trees and ground mositure into our ABM model. For the environment class the 'init' function differs from both the moss and weed class. The new 'init' function initialises the number of trees and their radius, this allows the next function 'place trees' to take the number of trees (if greater than zero) and randomise an x and y coordinate and sets it equal to 2. The function then uses a for loop to iterate over all of the cells within the tree's (circle's) radius and sets two conidition: if i**2 + j**2 > (self.tree_radius-1)**2 is true then self.environ[x][y] = 4 else: self.environ[x][y] = 2...

## References